#### Задание на ПР-8:

1. Сгенерируйте неразмеченный датасет с данными о студентах: направление подготовки (одно из
трех возможных: 090301, 090304 или 100503), наличие задолженности за прошлую сессию (да/нет), наличие задолженности за более ранние сессии (да/нет), класс пропусков за истекший семестр (посетил менее 30%, от 30% до 50%, от 50% до 80%, более 80%), зарегистрирован в беседах ВК кафедры (да/нет), зарегистрирован в moodle по всем дисциплинам (да/нет), спортсмен (да/нет), активен в студенческих мероприятиях (да/нет), отзывы преподавателей (плохо / хорошо / отлично). 
2. Проставьте метки True (продолжает обучение) / False (не продолжает обучение). Либо программно (на защите расскажите как сделали), либо вручную. Случайно расставлять метки запрещено - в данных должна быть закономерность.
3. Разделите данные на обучающие и тестовые.
4. Постройте ДПР методом ID3 на обучающих данных.
5. Проведите классификацию на тестовых данных.
6. Оцените результат с помощью метрик precision и recall

In [1]:
# Импорт необходимых библиотек
import random  # Для генерации случайных чисел и выбора случайных элементов
import math    # Для математических операций (логарифм для энтропии)
from collections import Counter  # Для работы с подсчетом элементов

#### 1. ГЕНЕРАЦИЯ ДАТАСЕТА

In [2]:
# Функция для генерации одного студента
def generate_student():
    # Список возможных направлений подготовки
    majors = ['090301', '090304', '100503']
    
    # Генерируем случайное направление подготовки
    major = random.choice(majors)
    
    # Генерируем наличие задолженности за текущую сессию (да/нет)
    current_debt = random.choice(['да', 'нет'])
    
    # Генерируем наличие задолженности за предыдущие сессии (да/нет)
    previous_debt = random.choice(['да', 'нет'])
    
    # Список возможных классов пропусков
    attendance_classes = ['менее 30%', 'от 30% до 50%', 'от 50% до 80%', 'более 80%']
    
    # Генерируем случайный класс пропусков
    attendance = random.choice(attendance_classes)
    
    # Генерируем регистрацию в беседах ВК (да/нет)
    vk_registered = random.choice(['да', 'нет'])
    
    # Генерируем регистрацию в moodle (да/нет)
    moodle_registered = random.choice(['да', 'нет'])
    
    # Генерируем статус спортсмена (да/нет)
    is_athlete = random.choice(['да', 'нет'])
    
    # Генерируем активность в мероприятиях (да/нет)
    is_active = random.choice(['да', 'нет'])
    
    # Список возможных отзывов преподавателей
    reviews = ['плохо', 'хорошо', 'отлично']
    
    # Генерируем случайный отзыв
    review = random.choice(reviews)
    
    # Возвращаем словарь с данными студента
    return {
        'major': major,
        'current_debt': current_debt,
        'previous_debt': previous_debt,
        'attendance': attendance,
        'vk_registered': vk_registered,
        'moodle_registered': moodle_registered,
        'is_athlete': is_athlete,
        'is_active': is_active,
        'review': review
    }

# Функция для определения метки (продолжает обучение или нет)
def assign_label(student):
    # Создаем логику отчисления на основе комбинации факторов
    
    # Высокий риск отчисления если есть текущие и предыдущие задолженности
    if student['current_debt'] == 'да' and student['previous_debt'] == 'да':
        return random.random() < 0.2
    
    # Высокий риск отчисления при очень плохой посещаемости и плохих отзывах
    if student['attendance'] == 'менее 30%' and student['review'] == 'плохо':
        return random.random() < 0.4
    
    # Риск отчисления если не зарегистрирован в системах и очень плохая посещаемость
    if (student['vk_registered'] == 'нет' and 
        student['moodle_registered'] == 'нет' and 
        student['attendance'] == 'менее 30%'):  
        return False
    
    # Риск отчисления при плохих отзывах и низкой активности
    if (student['review'] == 'плохо' and 
        student['is_athlete'] == 'нет' and 
        student['is_active'] == 'нет'):
        return False
    
    # Риск отчисления при плохой посещаемости и отсутствии в moodle
    if (student['attendance'] in ['менее 30%', 'от 30% до 50%'] and  # Плохая посещаемость
        student['moodle_registered'] == 'нет'):
        return False
    
    # Во всех остальных случаях студент продолжает обучение
    return True

In [3]:
# Генерация датасета из 500 студентов

# Создаем пустой список для хранения данных студентов
students = []

# Создаем пустой список для хранения меток (продолжает/не продолжает)
labels = []

# Цикл для генерации 500 студентов
for i in range(500):
    # Генерируем данные одного студента
    student = generate_student()
    
    # Определяем метку для студента
    label = assign_label(student)
    
    # Добавляем данные студента в список
    students.append(student)
    
    # Добавляем метку в список
    labels.append(label)

# Выводим информацию о размере датасета
print(f"Сгенерировано студентов: {len(students)}")
print(f"Количество меток: {len(labels)}")

Сгенерировано студентов: 500
Количество меток: 500


### Вероятность класса в функции class_probabilities вычисляется по формуле:

$$p_i = \frac{\text{количество элементов класса } i}{\text{общее количество элементов}}$$

**где:**
- $p_i$ - вероятность i-го класса
- $\text{количество элементов класса } i$ - число элементов с данной меткой
- $\text{общее количество элементов}$ - общий размер набора данных

---

### Энтропия вычисляется по формуле Шеннона в функции entropy:

$$H(S) = -\sum_{i=1}^{n} p_i \log_2 p_i$$

**где:**
- $H(S)$ - энтропия множества S
- $p_i$ - вероятность i-го класса в множестве S
- $n$ - количество классов
- $\log_2$ - логарифм по основанию 2

---

### Вычисление общей энтропии разделения по одному атрибуту в функции partition_entropy по формуле:
$H(S) = q_1H(S_1) + \cdots + q_mH(S_m)$

где:

$q_j = \frac{|S_j|}{|S|}$ - вес (коэффициент) j-го подмножества

$H(S_j)$ - энтропия j-го подмножества

In [4]:
# Функция для вычисления вероятностей классов
def class_probabilities(labels):
    # Подсчитываем общее количество меток
    total_count = len(labels)
    # Создаем счетчик для подсчета количества каждой метки
    counts = Counter(labels)
    # Создаем пустой список для вероятностей
    probabilities = []
    # Для каждого значения счетчика вычисляем вероятность
    for count in counts.values():
        # Вычисляем вероятность и добавляем в список
        probability = count / total_count
        probabilities.append(probability)
    # Возвращаем список вероятностей
    return probabilities

# Функция для вычисления энтропии
def entropy(class_probabilities):
    # Инициализируем энтропию
    result = 0
    # Для каждой вероятности в списке
    for p in class_probabilities:
        # Если вероятность больше нуля (избегаем log(0))
        if p > 0:
            # Добавляем вклад текущей вероятности в энтропию
            result -= p * math.log2(p)
    # Возвращаем вычисленную энтропию
    return result

# Функция для вычисления энтропии размеченных данных
def data_entropy(labeled_data):
    # labeled_data имеет формат [(данные, метка), ...]
    # Извлекаем все метки из размеченных данных
    labels = []
    for item, label in labeled_data:
        labels.append(label)
    # Вычисляем вероятности классов
    probabilities = class_probabilities(labels)
    # Возвращаем энтропию вычисленных вероятностей
    return entropy(probabilities)

# Функция для вычисления общей энтропии разделения по одному атрибуту
def partition_entropy(subsets):
    # Вычисляем общее количество элементов во всех подмножествах
    total_count = 0
    for subset in subsets:
        total_count += len(subset)
    
    # Инициализируем общую энтропию
    result = 0
    # Для каждого подмножества в разделении
    for subset in subsets:
        # Если подмножество не пустое
        if len(subset) > 0:
            # Вычисляем вес (коэффициент) подмножества
            weight = len(subset) / total_count
            # Вычисляем энтропию подмножества
            subset_entropy = data_entropy(subset)
            # Добавляем взвешенную энтропию к общей
            result += weight * subset_entropy
    # Возвращаем общую энтропию разделения
    return result

#### 2. РАЗДЕЛЕНИЕ ДАННЫХ НА ОБУЧАЮЩИЕ И ТЕСТОВЫЕ

In [5]:
# Функция для разделения данных по атрибуту
def partition_by(inputs, attribute):
    # inputs - данные в формате [(данные_студента, метка), ...] 
    
    # Создаем словарь для групп с разделенными данными
    groups = {}
    # Для каждого элемента входных данных
    for inp in inputs:
        # Извлекаем значение атрибута
        key = inp[0][attribute]
        
        # Если ключа еще нет в словаре, создаем пустой список
        if key not in groups:
            groups[key] = []
        
        # Добавляем элемент в соответствующую группу
        groups[key].append(inp)
    
    # Возвращаем словарь с разделенными данными , где ключи - значения атрибута, значения - списки элементов, имеющих это значение
    return groups

# Функция для вычисления энтропии разделения по атрибуту
def partition_entropy_by(inputs, attribute):
    # Разделяем данные по указанному атрибуту
    partitions = partition_by(inputs, attribute)
    # Вычисляем энтропию разделения
    return partition_entropy(partitions.values()) # partitions.values() - список списков, где внутренний список - список данных

#### 3. РЕАЛИЗАЦИЯ АЛГОРИТМА ID3

ДПР будет состоять из решающих узлов, в которых задается вопрос и после ответа указывается нужное направление, в которых задается вопроса и после ответа указывается нужное направление, а также листовых узлов, в которых озвучивается прогноз. 

**Алгоритм ID3 имеет вид:** 

1. Если все данные имеют одинаковую метку, то создать листовой узел, предсказывающий эту метку, и остановиться. 
2. Если список атрибутов пуст (нет вопросов), то создать листовой узел, предсказывающий наиболее общую метку, и остановиться. 
3. Разбить данные по каждому атрибуту. 
4. Выбрать разбиение (и соотвествующий атрибут) с наименьшей энтропией. 
5. Добавить решающий узел с выбранным атрибутом (значением признака). 
6. Повторить для каждого подмножества разбиения.

In [6]:
# Функция для построения дерева решений методом ID3
def build_tree_id3(inputs, split_candidates=None):
    # Если кандидаты для разделения не заданы, используем все атрибуты из первого элемента
    if split_candidates is None:
        split_candidates = list(inputs[0][0].keys())
    
    # Подсчитываем общее количество элементов
    num_inputs = len(inputs)
    
    # Подсчитываем количество True меток
    num_trues = 0
    for item, label in inputs:
        if label:
            num_trues += 1
    
    # Подсчитываем количество False меток
    num_falses = num_inputs - num_trues
    
    # Случай 1: если все метки False - создаем лист "ОТЧИСЛЕН"
    if num_trues == 0:
        return False
    
    # Случай 2: если все метки True - создаем лист "ПРОДОЛЖАЕТ"
    if num_falses == 0: 
        return True
    
    # Случай 3: если не осталось кандидатов (атрибутов) для разделения
    if not split_candidates:
        # Возвращаем наиболее частую метку
        return num_trues >= num_falses
    
    # Выбираем лучший атрибут для разделения (с минимальной энтропией)
    # Создаем вспомогательную функцию для вычисления энтропии по атрибуту
    def get_entropy(attribute):
        return partition_entropy_by(inputs, attribute)
    
    # Находим атрибут с минимальной энтропией
    best_attribute = None
    best_entropy = float('inf')
    
    for attribute in split_candidates:
        current_entropy = get_entropy(attribute)
        if current_entropy < best_entropy:
            best_entropy = current_entropy
            best_attribute = attribute
    
    # Разделяем данные по лучшему атрибуту
    partitions = partition_by(inputs, best_attribute)
    
    # Создаем новый список кандидатов без использованного атрибута
    new_candidates = []
    for a in split_candidates:
        if a != best_attribute:
            new_candidates.append(a)

    # Создаем пустой словарь для хранения поддеревьев
    subtrees = {}
    # Перебираем все пары (значение_атрибута, подмножество_данных) из разделения
    for attribute_value, subset in partitions.items():
        # Рекурсивно строим поддеревья для каждого значения атрибута
        subtrees[attribute_value] = build_tree_id3(subset, new_candidates)
    
    # Добавляем значение по умолчанию (наиболее частая метка в текущем узле)
    subtrees[None] = num_trues > num_falses
    
    # Возвращаем узел дерева - кортеж (атрибут, словарь поддеревьев)
    return (best_attribute, subtrees)

# Функция для классификации с помощью построенного дерева
def classify(tree, inp):
    # inp - это словарь с данными одного студента
    # Базовый случай: если дерево является листом (True или False)
    if tree in (True, False):
        return tree
    
    # Извлекаем атрибут и словарь поддеревьев
    attribute, subtree_dict = tree
    # Получаем значение атрибута из входных данных
    subtree_key = inp.get(attribute)
    
    # Если значения нет в словаре поддеревьев, используем значение по умолчанию
    if subtree_key not in subtree_dict:
        subtree_key = None
    
    # Получаем соответствующее поддерево
    subtree = subtree_dict[subtree_key]
    # Рекурсивно классифицируем с помощью поддерева
    return classify(subtree, inp)

In [7]:
# Подготовка данных в формате, требуемом алгоритмом ID3

# Создаем объединенный список размеченных данных
labeled_data = []
# Объединяем данные студентов и метки в один список кортежей
for i in range(len(students)):
    labeled_data.append((students[i], labels[i]))

# Разделяем данные на обучающие и тестовые в правильном формате
train_data = labeled_data[:400]  # Первые 400 элементов для обучения
test_data = labeled_data[400:]   # Последние 100 элементов для тестирования

# Выводим информацию о данных
print(f"Всего размеченных данных: {len(labeled_data)}")
print(f"Обучающая выборка: {len(train_data)}")
print(f"Тестовая выборка: {len(test_data)}")

Всего размеченных данных: 500
Обучающая выборка: 400
Тестовая выборка: 100


In [8]:
# Анализ энтропии разбиения по каждому атрибуту
print("=" * 60)
print("АНАЛИЗ ЭНТРОПИИ ПО АТРИБУТАМ (на обучающих данных)")
print("=" * 60)

# Список всех атрибутов
attributes = list(train_data[0][0].keys())

# Вычисляем энтропию для каждого атрибута
entropy_results = []
for attribute in attributes:
    entropy_val = partition_entropy_by(train_data, attribute)
    entropy_results.append((attribute, entropy_val))
    print(f"Атрибут: {attribute:20} Энтропия: {entropy_val:.4f}")

# Находим атрибут с минимальной энтропией (лучший для разделения)
best_attr, best_entropy = min(entropy_results, key=lambda x: x[1])
print("=" * 60)
print(f"ЛУЧШИЙ АТРИБУТ ДЛЯ РАЗДЕЛЕНИЯ: '{best_attr}' с энтропией {best_entropy:.4f}")
print("=" * 60)

АНАЛИЗ ЭНТРОПИИ ПО АТРИБУТАМ (на обучающих данных)
Атрибут: major                Энтропия: 0.9743
Атрибут: current_debt         Энтропия: 0.9484
Атрибут: previous_debt        Энтропия: 0.8995
Атрибут: attendance           Энтропия: 0.8505
Атрибут: vk_registered        Энтропия: 0.9749
Атрибут: moodle_registered    Энтропия: 0.9341
Атрибут: is_athlete           Энтропия: 0.9627
Атрибут: is_active            Энтропия: 0.9728
Атрибут: review               Энтропия: 0.9654
ЛУЧШИЙ АТРИБУТ ДЛЯ РАЗДЕЛЕНИЯ: 'attendance' с энтропией 0.8505


#### 4. ПОСТРОЕНИЕ И ТЕСТИРОВАНИЕ МОДЕЛИ

In [9]:
# Функция для вывода структуры дерева решений
def print_tree_structure(tree, indent="", depth=0, max_depth=3):
    """
    Рекурсивная функция для вывода структуры дерева
    """
    if depth > max_depth:
        print(indent + "  ... (глубина ограничена)")
        return
        
    # Если узел является листом
    if tree in (True, False):
        result = "ПРОДОЛЖАЕТ" if tree else "ОТЧИСЛЕН"
        print(indent + f"Лист: {result}")
        return
    
    # Если узел является решающим
    attribute, subtrees = tree
    print(indent + f"[Уровень {depth}] Атрибут: '{attribute}'")
    
    # Выводим все ветви этого узла
    for value, subtree in subtrees.items():
        if value is not None:
            print(indent + f"  ├── Если '{attribute}' = '{value}':")
            print_tree_structure(subtree, indent + "  │     ", depth + 1, max_depth)
        else:
            print(indent + f"  └── По умолчанию:")
            print_tree_structure(subtree, indent + "        ", depth + 1, max_depth)

In [10]:
# Построение дерева решений с помощью алгоритма ID3
print("Начинаем построение дерева решений методом ID3...")
decision_tree = build_tree_id3(train_data)

# Выводим информацию о корневом узле дерева
if isinstance(decision_tree, tuple):
    root_attribute, subtrees = decision_tree
    print(f"Построено дерево решений")
    print(f"Корневой атрибут: '{root_attribute}'")
    print(f"Количество ветвей: {len(subtrees)}")
else:
    print(f"Дерево состоит из одного листа: {decision_tree}")

# Выводим структуру дерева (первые 3 уровня)
print("\n" + "=" * 50)
print("СТРУКТУРА ДЕРЕВА РЕШЕНИЙ (первые 3 уровня)")
print("=" * 50)
print_tree_structure(decision_tree, max_depth=3)

# Создаем пустой список для предсказаний
predictions = []

# Классифицируем каждый образец в тестовой выборке
for sample, true_label in test_data:
    # Классифицируем образец с помощью дерева
    prediction = classify(decision_tree, sample)
    # Добавляем предсказание в список
    predictions.append(prediction)

# Извлекаем истинные метки из тестовых данных
true_labels = []
for sample, label in test_data:
    true_labels.append(label)

# Выводим информацию о классификации
print(f"Построено дерево решений методом ID3")
print(f"Классифицировано тестовых образцов: {len(predictions)}")

Начинаем построение дерева решений методом ID3...
Построено дерево решений
Корневой атрибут: 'attendance'
Количество ветвей: 5

СТРУКТУРА ДЕРЕВА РЕШЕНИЙ (первые 3 уровня)
[Уровень 0] Атрибут: 'attendance'
  ├── Если 'attendance' = 'менее 30%':
  │     [Уровень 1] Атрибут: 'moodle_registered'
  │       ├── Если 'moodle_registered' = 'да':
  │       │     [Уровень 2] Атрибут: 'current_debt'
  │       │       ├── Если 'current_debt' = 'нет':
  │       │       │     [Уровень 3] Атрибут: 'review'
  │       │       │       ├── Если 'review' = 'хорошо':
  │       │       │       │       ... (глубина ограничена)
  │       │       │       ├── Если 'review' = 'плохо':
  │       │       │       │       ... (глубина ограничена)
  │       │       │       ├── Если 'review' = 'отлично':
  │       │       │       │       ... (глубина ограничена)
  │       │       │       └── По умолчанию:
  │       │       │               ... (глубина ограничена)
  │       │       ├── Если 'current_debt' = 'да':
  │  

In [11]:
# Демонстрация процесса классификации для нескольких примеров
print("\n" + "=" * 60)
print("ПРИМЕРЫ КЛАССИФИКАЦИИ С ДЕТАЛИЗАЦИЕЙ")
print("=" * 60)

# Функция для детальной классификации с выводом пути
def classify_detailed(tree, inp, path=None):
    if path is None:
        path = []
    
    # Базовый случай: листовой узел
    if tree in (True, False):
        result = "ПРОДОЛЖАЕТ" if tree else "ОТЧИСЛЕН"
        path.append(f"РЕЗУЛЬТАТ: {result}")
        return tree, path
    
    # Решающий узел
    attribute, subtree_dict = tree
    value = inp.get(attribute)
    
    path.append(f"Атрибут: '{attribute}' = '{value}'")
    
    # Выбираем поддерево
    if value in subtree_dict:
        subtree = subtree_dict[value]
    else:
        subtree = subtree_dict[None]
        path[-1] += " (значение по умолчанию)"
    
    # Рекурсивно продолжаем классификацию
    return classify_detailed(subtree, inp, path)

# Показываем примеры классификации для 3 тестовых случаев
print("Примеры процесса классификации:\n")
for i in range(3):
    sample, true_label = test_data[i]
    true_result = "ПРОДОЛЖАЕТ" if true_label else "ОТЧИСЛЕН"
    
    print(f"Пример {i+1}:")
    print(f"  Данные студента: {sample}")
    prediction, decision_path = classify_detailed(decision_tree, sample)
    predicted_result = "ПРОДОЛЖАЕТ" if prediction else "ОТЧИСЛЕН"
    
    print(f"  Путь классификации:")
    for step in decision_path:
        print(f"    {step}")
    
    status = "ВЕРНО" if prediction == true_label else "ОШИБКА"
    print(f"  Результат: {predicted_result} (истина: {true_result}) {status}")
    print()


ПРИМЕРЫ КЛАССИФИКАЦИИ С ДЕТАЛИЗАЦИЕЙ
Примеры процесса классификации:

Пример 1:
  Данные студента: {'major': '100503', 'current_debt': 'нет', 'previous_debt': 'да', 'attendance': 'от 50% до 80%', 'vk_registered': 'нет', 'moodle_registered': 'да', 'is_athlete': 'нет', 'is_active': 'нет', 'review': 'плохо'}
  Путь классификации:
    Атрибут: 'attendance' = 'от 50% до 80%'
    Атрибут: 'previous_debt' = 'да'
    Атрибут: 'current_debt' = 'нет'
    Атрибут: 'is_active' = 'нет'
    Атрибут: 'moodle_registered' = 'да'
    РЕЗУЛЬТАТ: ПРОДОЛЖАЕТ
  Результат: ПРОДОЛЖАЕТ (истина: ОТЧИСЛЕН) ОШИБКА

Пример 2:
  Данные студента: {'major': '100503', 'current_debt': 'нет', 'previous_debt': 'да', 'attendance': 'более 80%', 'vk_registered': 'да', 'moodle_registered': 'нет', 'is_athlete': 'нет', 'is_active': 'да', 'review': 'отлично'}
  Путь классификации:
    Атрибут: 'attendance' = 'более 80%'
    Атрибут: 'previous_debt' = 'да'
    Атрибут: 'current_debt' = 'нет'
    Атрибут: 'review' = 'отлично'
  

#### 5. ОЦЕНКА МОДЕЛИ С ПОМОЩЬЮ МЕТРИК PRECISION И RECALL

In [12]:
# Функция для вычисления метрик качества классификации
def evaluate_metrics(true_labels, predicted_labels):
    # Инициализация счетчиков
    true_positive = 0  # Правильно предсказанные True
    false_positive = 0 # Неправильно предсказанные True
    true_negative = 0  # Правильно предсказанные False
    false_negative = 0 # Неправильно предсказанные False
    
    # Подсчет TP, FP, TN, FN
    for i in range(len(true_labels)):
        if true_labels[i] and predicted_labels[i]:
            true_positive += 1
        elif not true_labels[i] and predicted_labels[i]:
            false_positive += 1
        elif not true_labels[i] and not predicted_labels[i]:
            true_negative += 1
        elif true_labels[i] and not predicted_labels[i]:
            false_negative += 1
    
    # Вычисление precision (точности)
    if true_positive + false_positive > 0:
        precision = true_positive / (true_positive + false_positive)
    else:
        precision = 0
    
    # Вычисление recall (полноты)
    if true_positive + false_negative > 0:
        recall = true_positive / (true_positive + false_negative)
    else:
        recall = 0
    
    # Вычисление accuracy (точности)
    accuracy = (true_positive + true_negative) / len(true_labels)
    
    # Возвращаем вычисленные метрики
    return precision, recall, accuracy, true_positive, false_positive, true_negative, false_negative

# Вычисляем метрики precision и recall
precision, recall, accuracy, tp, fp, tn, fn = evaluate_metrics(true_labels, predictions)

# Выводим результаты
print("=" * 60)
print("ОЦЕНКА КАЧЕСТВА КЛАССИФИКАЦИИ")
print("=" * 60)
print(f"Precision (Точность): {precision:.4f}")
print(f"Recall (Полнота):    {recall:.4f}")
print(f"Accuracy (Правильность): {accuracy:.4f}")
print()
print("Матрица ошибок:")
print(f"  Истинно-положительные (TP): {tp}")
print(f"  Ложно-положительные (FP):   {fp}")
print(f"  Истинно-отрицательные (TN): {tn}")
print(f"  Ложно-отрицательные (FN):   {fn}")
print()
print(f"Размер обучающей выборки: {len(train_data)}")
print(f"Размер тестовой выборки:  {len(test_data)}")

ОЦЕНКА КАЧЕСТВА КЛАССИФИКАЦИИ
Precision (Точность): 0.9200
Recall (Полнота):    0.7931
Accuracy (Правильность): 0.8400

Матрица ошибок:
  Истинно-положительные (TP): 46
  Ложно-положительные (FP):   4
  Истинно-отрицательные (TN): 38
  Ложно-отрицательные (FN):   12

Размер обучающей выборки: 400
Размер тестовой выборки:  100
